# Muhammad Tahir Zia (2021465)
# Akhtar Ali (2021758)

## Using MarianMT [Helsinki mt-opus-ur-en](https://huggingface.co/Helsinki-NLP/opus-mt-ur-en) on HugginFace as a pre-trained model


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


We will **fine-tune** one of the **hugging-face Transformers model** for translating **Urdu** to **English** language.

## Install required packages

In [ ]:
!pip install datasets transformers sacrebleu torch sentencepiece transformers[sentencepiece]

## Importing Libraries

In [ ]:
!pip install evaluate

In [ ]:
import transformers
from datasets import Dataset
import evaluate
from google.colab import drive

# Example metric load
metric = evaluate.load("accuracy")

print(transformers.__version__)

4.51.3


## Define Model Checkpoint

In [ ]:
model_checkpoint = "Helsinki-NLP/opus-mt-ur-en"

## Get Current Working Directory

In [ ]:
import os
my_dir = '/content/drive/MyDrive/huggingface'
print(my_dir)

/content/drive/MyDrive/huggingface


## Loading the dataset

In [ ]:
# Path to translation file
path_to_data = '/content/drive/MyDrive/huggingface/urd.txt'

# Read file
translation_file = open(path_to_data,"r", encoding='utf-8')
raw_data = translation_file.read()
translation_file.close()

# Parse data
raw_data = raw_data.split('\n')
ur_en_pairs = [sentence.split('\t') for sentence in  raw_data]
del(ur_en_pairs[1143])

## Creating Custom DataSet

In [ ]:
training_dataset = ur_en_pairs[:1000]
validation_dataset = ur_en_pairs[1000:1100]
testing_dataset = ur_en_pairs[1100:]

In [ ]:
train_dict = [{'ur': i[1], 'en': i[0]} for i in training_dataset]
test_dict = [{'ur': i[1], 'en': i[0]} for i in testing_dataset]
validate_dict = [{'ur': i[1], 'en': i[0]} for i in validation_dataset]

In [ ]:
train_dict = {'translation': train_dict}
test_dict = {'translation': test_dict}
validate_dict = {'translation': validate_dict}

In [ ]:
train_dataset = Dataset.from_dict(train_dict)
test_dataset = Dataset.from_dict(test_dict)
validate_dataset = Dataset.from_dict(validate_dict)


In [ ]:
import transformers
from datasets import Dataset
import evaluate
from google.colab import drive
import os
from datasets import DatasetDict


In [ ]:
final_dataset = DatasetDict({"train":train_dataset, "validation":validate_dataset, "test":test_dataset})

In [ ]:
final_dataset

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 100
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 43
    })
})

In [ ]:
final_dataset['train'][855:865]

{'translation': [{'en': "You don't need to go in such a hurry.",
   'ur': 'تمھیں اتنی جلدی جانے کی ضرورت نہیں ہے۔'},
  {'en': 'Your mother will be back before long.',
   'ur': 'تمہاری امی بس آتی ہونگی۔'},
  {'en': 'Your mother will be back before long.',
   'ur': 'تمہاری ماں بس راستے میں ہو گی۔'},
  {'en': 'All of us were busy cleaning the room.',
   'ur': 'ہم سارے کمرے کی صفائی کرنے میں مصروف تھے۔'},
  {'en': 'All you have to do is sweep the floor.',
   'ur': 'تمہیں صرف زمین کی صفائی کرنی ہے۔'},
  {'en': 'Different people have different ideas.',
   'ur': 'مختلف لوگوں کے مختلف خیالات ہیں۔'},
  {'en': 'English is studied all over the world.',
   'ur': 'انگریزی پوری دنیا میں پڑھای جاتی ہے۔'},
  {'en': 'Everyone was listening very carefully.',
   'ur': 'سارے بڑے غور سے سن رہے تھے۔'},
  {'en': 'He does not take care of his children.',
   'ur': 'وہ اپنے بچوں کا خیال نہیں کرتا۔'},
  {'en': 'I have a lot of passwords to remember.',
   'ur': 'مجھے کافی زیادہ پاسورڈس یاد دکھنے پڑتے ہے۔'}]}

To get a sense of how the data looks like, the following function will show some examples picked randomly in the dataset.

In [ ]:
import random
import pandas as pd
from IPython.display import display, HTML
import datasets

def show_random_elements(dataset, num_examples=5):

    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."

    picks = []

    for _ in range(num_examples):

        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])

    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

show_random_elements(final_dataset["train"])

,translation
0,"{'en': 'Tom is spraying his garden.', 'ur': 'ٹام اپنے باغ کو پانی دے رہا ہے۔'}"
1,"{'en': 'Tom's trapped.', 'ur': 'ٹام پھنسا ہوا ہے۔'}"
2,"{'en': 'In hot weather, water evaporates quickly.', 'ur': 'گرم موسم میں پانی جلد اڑ جاتا ہے۔'}"
3,"{'en': 'They don't take care of that dog.', 'ur': 'وہ اس کتے کا خیال نہیں رکھتے۔'}"
4,"{'en': 'Please come. I really want to meet you.', 'ur': 'پلیز آجائو۔ میں تم سے ملنے کے لیئے بیتاب ہوں۔'}"


We will use **metric** to evaluation (to compare our model to the benchmark). This can be easily done with the functions **load_metric**.

In [ ]:
from evaluate import load as load_metric  # New way

In [ ]:
metric = load_metric("sacrebleu")

In [ ]:
metric

EvaluationModule(name: "sacrebleu", module_type: "metric", features: [{'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id='references')}, {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}], usage: """
Produces BLEU scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions (`list` of `str`): list of translations to score. Each translation should be tokenized into a list of tokens.
    references (`list` of `list` of `str`): A list of lists of references. The contents of the first sub-list are the references for the first prediction, the contents of the second sub-list are for the second prediction, etc. Note that there must be the same number of references for each prediction (i.e. all sub-lists must be of the same length).
    smooth_method (`str`): The smoothing method to use, defaults to `'e

You can call its **compute method** with your **predictions** and **labels**, which need to be **list of decoded strings** (list of list for the labels):

In [ ]:
fake_preds = ["سنو ذرا", "hello there"]
fake_labels = [["سنو ذرا"], ["hello there"]]
metric.compute(predictions=fake_preds, references=fake_labels,)

{'score': 0.0,
 'counts': [4, 2, 0, 0],
 'totals': [4, 2, 0, 0],
 'precisions': [100.0, 100.0, 0.0, 0.0],
 'bp': 1.0,
 'sys_len': 4,
 'ref_len': 4}

## Preprocessing (Tokenization) the dataset

Before we can feed those texts to our model, we need to preprocess them. This is done by a **Transformers Tokenizer**

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/848k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/816k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
tokenizer.save_pretrained(my_dir)

('/content/drive/MyDrive/huggingface/tokenizer_config.json',
 '/content/drive/MyDrive/huggingface/special_tokens_map.json',
 '/content/drive/MyDrive/huggingface/vocab.json',
 '/content/drive/MyDrive/huggingface/source.spm',
 '/content/drive/MyDrive/huggingface/target.spm',
 '/content/drive/MyDrive/huggingface/added_tokens.json')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(my_dir)

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


You can directly call this tokenizer on one sentence or a pair of sentences:

In [ ]:
tokenizer(["Hello, this one sentence!", "This is another sentence."])

{'input_ids': [[56936, 9004, 2, 19741, 2889, 73, 4695, 73, 66, 7061, 15423, 67, 0], [4117, 15061, 66, 22, 73, 55163, 3565, 73, 66, 7061, 15423, 5, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

To prepare the **targets for our model**, we need to tokenize them inside the **as_target_tokenizer** context manager.

In [ ]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["Hello, this one sentence!", "This is another sentence."]))

{'input_ids': [[20475, 2, 140, 155, 7166, 67, 0], [416, 22, 474, 7166, 5, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
print("Or")

tgt_text = ["Hello, this one sentence!", "This is another sentence."],
print(tokenizer(text_target = tgt_text))

Or
{'input_ids': [[20475, 2, 140, 155, 7166, 67, 416, 22, 474, 7166, 5, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


This Function will **preprocess our samples**.


In [ ]:
prefix = ""

max_input_length = 128
max_target_length = 128

source_lang = "ur"
target_lang = "en"

def preprocess_function(examples):
    """
    Getting Input and target
    """
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]

    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

In [ ]:
preprocess_function(final_dataset['train'][:2])

{'input_ids': [[31585, 0], [349, 412, 0]], 'attention_mask': [[1, 1], [1, 1, 1]], 'labels': [[16918, 5, 0], [5569, 67, 0]]}

To **apply** this function on **all** the pairs of sentences in our dataset, we just use the **map** method of our **dataset** object we created earlier.

In [ ]:
tokenized_datasets = final_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/43 [00:00<?, ? examples/s]

## Fine-tuning the model
Now that our data is ready, we can **download the pretrained** model and fine-tune it.

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

pytorch_model.bin:   0%|          | 0.00/306M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/306M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

## Saving the Model

In [ ]:
model.save_pretrained(my_dir+"save_model")

/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[62024]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


## Loading Model from Save Directory

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained('/content/drive/MyDrive/huggingfacesave_model')

To **instantiate** a **Seq2SeqTrainer**, we will need to define **three** more things. The most important is the **Seq2SeqTrainingArguments**.

In [ ]:
batch_size = 16

model_name = model_checkpoint.split("/")[-1]
print("Model name:", model_name)

args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    # The 'evaluation_strategy' argument has been replaced with 'eval_strategy'
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=20,
    predict_with_generate=True
)

Model name: opus-mt-ur-en


In [ ]:
args

Seq2SeqTrainingArguments(output_dir='opus-mt-ur-en-finetuned-ur-to-en', overwrite_output_dir=False, do_train=False, do_eval=True, do_predict=False, eval_strategy=<IntervalStrategy.EPOCH: 'epoch'>, prediction_loss_only=False, per_device_train_batch_size=16, per_device_eval_batch_size=16, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, eval_delay=0, torch_empty_cache_steps=None, learning_rate=2e-05, weight_decay=0.01, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=20, max_steps=-1, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, lr_scheduler_kwargs={}, warmup_ratio=0.0, warmup_steps=0, log_level='passive', log_level_replica='warning', log_on_each_node=True, logging_dir='opus-mt-ur-en-finetuned-ur-to-en/runs/May05_18-42-00_028037d79733', logging_strategy=<IntervalStrategy.STEPS: 'steps'>, logging_first_step=False, logging_steps=500, logging_nan_inf_filter=True, save_str

Here we **set** the **evaluation** to be done at the **end of each epoch**.

Then, we need a special kind of **data collator**, which will not only **pad** the **inputs** to the **maximum length** in the batch, but also the **labels**:

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

The last thing to define for our **Seq2SeqTrainer** is **how to compute the metrics** from the **predictions**. We need to define a function for this.

In [ ]:
import numpy as np

def postprocess_text(preds, labels):

    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels


def compute_metrics(eval_preds):

    preds, labels = eval_preds

    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions = decoded_preds, references = decoded_labels)

    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]

    result["gen_len"] = np.mean(prediction_lens)

    result = {k: round(v, 4) for k, v in result.items()}

    return result

Then we just need to pass all of this along with our datasets to the Seq2SeqTrainer:

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset = tokenized_datasets["train"],
    eval_dataset = tokenized_datasets["validation"],
    data_collator = data_collator,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics
)

<ipython-input-73-3fc49cbd6f34>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


We can now finetune our model by just calling the train method:

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: itstahir256 (itstahir256-ghulam-ishaq-khan-institute-of-engineering-s) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,No log,1.847535,20.894200,11.460000
2,No log,1.860234,20.552600,11.600000
3,No log,1.874768,20.349700,11.660000
4,No log,1.914582,20.166800,11.670000
5,No log,1.974658,19.909200,11.360000
6,No log,2.008621,20.243200,11.390000
7,No log,2.053040,20.408800,11.470000
8,0.860100,2.074333,20.777200,11.460000
9,0.860100,2.096689,20.824100,11.240000
10,0.860100,2.140495,20.754100,11.310000


TrainOutput(global_step=1260, training_loss=0.46567564767504493, metrics={'train_runtime': 5418.3026, 'train_samples_per_second': 3.691, 'train_steps_per_second': 0.233, 'total_flos': 70627243327488.0, 'train_loss': 0.46567564767504493, 'epoch': 20.0})

In [ ]:
trainer.save_model()

In [ ]:
import os
for dirname, _, filenames in os.walk('opus-mt-ur-en-finetuned-ur-to-en'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

opus-mt-ur-en-finetuned-ur-to-en/source.spm
opus-mt-ur-en-finetuned-ur-to-en/config.json
opus-mt-ur-en-finetuned-ur-to-en/special_tokens_map.json
opus-mt-ur-en-finetuned-ur-to-en/tokenizer_config.json
opus-mt-ur-en-finetuned-ur-to-en/vocab.json
opus-mt-ur-en-finetuned-ur-to-en/training_args.bin
opus-mt-ur-en-finetuned-ur-to-en/target.spm
opus-mt-ur-en-finetuned-ur-to-en/generation_config.json
opus-mt-ur-en-finetuned-ur-to-en/model.safetensors
opus-mt-ur-en-finetuned-ur-to-en/runs/May05_18-42-00_028037d79733/events.out.tfevents.1746470666.028037d79733.2142.0
opus-mt-ur-en-finetuned-ur-to-en/checkpoint-1260/source.spm
opus-mt-ur-en-finetuned-ur-to-en/checkpoint-1260/config.json
opus-mt-ur-en-finetuned-ur-to-en/checkpoint-1260/trainer_state.json
opus-mt-ur-en-finetuned-ur-to-en/checkpoint-1260/optimizer.pt
opus-mt-ur-en-finetuned-ur-to-en/checkpoint-1260/special_tokens_map.json
opus-mt-ur-en-finetuned-ur-to-en/checkpoint-1260/tokenizer_config.json
opus-mt-ur-en-finetuned-ur-to-en/checkpoi

Our fine tuned model already saved under opus-mt-ur-en-finetuned-ur-to-en/

Load the model and translate some text from english to romanian

In [ ]:
from transformers import MarianMTModel, MarianTokenizer

# Sentence to translate
src_text = ['میں رات کے کھانے سے پہلے سونا چاہتا ہوں']

model_name = 'opus-mt-ur-en-finetuned-ur-to-en'

tokenizer = MarianTokenizer.from_pretrained(model_name)

print(tokenizer.supported_language_codes)

[]


/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


The bare Marian Model outputting raw hidden-states without any specific head on top. This model inherits from PreTrainedModel.

In [ ]:
model = MarianMTModel.from_pretrained(model_name)

translated = model.generate(**tokenizer(src_text, return_tensors="pt", padding=True))

result = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]

In [ ]:
result

['I want to sleep before dinner.']

our fine tune model is doing far better than pre-trained model